<a href="https://colab.research.google.com/github/wlsgud623/vaccine_tweet_analysis/blob/main/VaccineTweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!pip install transformers==3

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import logging

logging.basicConfig(level=logging.ERROR)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

     |████████████████████████████████| 754 kB 11.3 MB/s 
     |████████████████████████████████| 895 kB 53.5 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 
     |████████████████████████████████| 3.0 MB 46.2 MB/s 
cuda:0


# Load Data

Kaggle API

In [2]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jinhyeoungbae","key":"dc42a77071a666d47d2e97f076322dfb"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download -d kaushiksuresh147/covidvaccine-tweets
! kaggle datasets download -d gpreda/all-covid19-vaccines-tweets
! kaggle datasets download -d cosmos98/twitter-and-reddit-sentimental-analysis-dataset

 92% 49.0M/53.1M [00:01<00:00, 27.8MB/s]
100% 53.1M/53.1M [00:01<00:00, 33.4MB/s]
 94% 28.0M/29.9M [00:01<00:00, 17.6MB/s]
100% 29.9M/29.9M [00:01<00:00, 30.8MB/s]
 50% 5.00M/10.0M [00:00<00:00, 14.7MB/s]
100% 10.0M/10.0M [00:00<00:00, 25.4MB/s]


In [5]:
!unzip all-covid19-vaccines-tweets.zip
!unzip covidvaccine-tweets.zip
!unzip twitter-and-reddit-sentimental-analysis-dataset.zip

Archive:  all-covid19-vaccines-tweets.zip
  inflating: vaccination_all_tweets.csv  
Archive:  covidvaccine-tweets.zip
  inflating: covidvaccine.csv        
Archive:  twitter-and-reddit-sentimental-analysis-dataset.zip
  inflating: Reddit_Data.csv         
  inflating: Twitter_Data.csv        


Kaggle Tweet Data

In [43]:
first_vaccine_data = pd.read_csv("/content/covidvaccine.csv", nrows = 50000)
second_vaccine_data = pd.read_csv("/content/vaccination_all_tweets.csv", nrows = 50000)

In [44]:
first_vaccine_data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64.0,11.0,110.0,False,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
1,Shubham Gupta,NaN,I will tell about all experiences of my life f...,14-08-2020 16:42,1.0,17.0,0.0,False,18-08-2020 12:55,#CoronavirusVaccine #CoronaVaccine #CovidVacci...,"['CoronavirusVaccine', 'CoronaVaccine', 'Covid...",Twitter for Android,False
2,Journal of Infectiology,NaN,Journal of Infectiology (ISSN 2689-9981) is ac...,14-12-2017 07:07,143.0,566.0,8.0,False,18-08-2020 12:46,Deaths due to COVID-19 in Affected Countries\n...,NaN,Twitter Web App,False
3,Zane,NaN,Fresher than you.,18-09-2019 11:01,29.0,25.0,620.0,False,18-08-2020 12:45,@Team_Subhashree @subhashreesotwe @iamrajchoco...,NaN,Twitter for Android,False
4,Ann-Maree O’Connor,"Adelaide, South Australia",Retired university administrator. Melburnian b...,24-01-2013 14:53,83.0,497.0,10737.0,False,18-08-2020 12:45,@michellegrattan @ConversationEDU This is what...,NaN,Twitter Web App,False


In [45]:
second_vaccine_data.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [46]:
data_column = ['user_name', 'user_description','user_location','user_followers','text']
first_vaccine_data = first_vaccine_data[data_column]
second_vaccine_data = second_vaccine_data[data_column]

In [47]:
print(first_vaccine_data['text'])

0        Australia to Manufacture Covid-19 Vaccine and ...
1        #CoronavirusVaccine #CoronaVaccine #CovidVacci...
2        Deaths due to COVID-19 in Affected Countries\n...
3        @Team_Subhashree @subhashreesotwe @iamrajchoco...
4        @michellegrattan @ConversationEDU This is what...
                               ...                        
49995    Anyone know of a good #CovidVaccine video in S...
49996    Britons who have received their #CovidVaccine ...
49997    #CovidVaccine If it's true, Exactly who is Bil...
49998    Thank you to #science &amp; all the #healthcar...
49999    When clarity and precision are most needed, go...
Name: text, Length: 50000, dtype: object


In [48]:
import re

def con(x):
  return ' '.join(re.sub("(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(x)).split())
    
first_vaccine_data['text']=first_vaccine_data['text'].apply(con)
second_vaccine_data['text']=second_vaccine_data['text'].apply(con)

In [49]:
print(first_vaccine_data['text'])
print(first_vaccine_data['text'][0])

0        Australia to Manufacture Covid 19 Vaccine and ...
1                             Australia is doing very good
2        Deaths due to COVID 19 in Affected Countries R...
3                           Subhashree Stay safe di amp da
4        This is what passes for leadership in our coun...
                               ...                        
49995    Anyone know of a good video in Spanish Trying ...
49996    Britons who have received their are to be offe...
49997    If it s true Exactly who is Bill Gates trying ...
49998    Thank you to amp all the workers on the ground...
49999    When clarity and precision are most needed gov...
Name: text, Length: 50000, dtype: object
Australia to Manufacture Covid 19 Vaccine and give it to the Citizens for free of cost AFP quotes Prime Minister


Train Data

In [13]:
#(0 = negative,  1 = neutral, 2 = positive)
test_data = pd.read_csv("/content/Twitter_Data.csv", nrows=5000)
test_data['category'] = test_data['category'] + 1
test_data.head() 


,clean_text,category
0,when modi promised “minimum government maximum...,0
1,talk all the nonsense and continue all the dra...,1
2,what did just say vote for modi welcome bjp t...,2
3,asking his supporters prefix chowkidar their n...,2
4,answer who among these the most powerful world...,2


# Pre-processing

In [15]:
#Tokenize Data
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

# Text DataSet

In [16]:
class GPReviewDataset(Dataset):
  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.reviews)
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [17]:
# 최대 토큰 길이
MAX_LEN = 120

In [18]:
df_train, df_test = train_test_split(
  test_data,
  test_size=0.1,
  random_state=RANDOM_SEED
)

df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(4500, 2)
(250, 2)
(250, 2)


In [19]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df['clean_text'].to_numpy(),
    targets=df['category'].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers = 2
  )

In [20]:
print(df_train.columns)

Index(['clean_text', 'category'], dtype='object')


In [21]:
BATCH_SIZE = 8

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

# Import Model

In [22]:
bert_model = BertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [23]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [24]:
class_names = ['negative', 'neutral', 'positive']

model = SentimentClassifier(len(class_names))
model = model.to(device)

In [25]:
data = next(iter(train_data_loader))

input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length
print(input_ids[0])
print()

torch.Size([8, 120])
torch.Size([8, 120])
tensor([  101,  1107,  7168,  1209, 19004,  1216,  5471,  7042,  9468,  3702,
         1144,  1694,  1917,  1111,  2869,  1234,  1107,  7168,  1155,  1934,
         9319, 12006,  1132,  1111,  2869,  1234,   182,  5412,  1182,  1144,
         7894,  1105,  7042,  1444,  1359, 12006,  1111,  2869,  1234,  1443,
         1251,  7812, 11245,  2394,  5067,  5789,  1194,   173, 21238,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       device='cuda:0

In [26]:
F.softmax(model(input_ids, attention_mask), dim=1)

tensor([[0.4496, 0.2408, 0.3097],
        [0.2476, 0.5124, 0.2400],
        [0.5099, 0.2027, 0.2874],
        [0.2877, 0.3455, 0.3668],
        [0.4964, 0.2637, 0.2399],
        [0.4301, 0.2160, 0.3538],
        [0.5010, 0.2304, 0.2686],
        [0.3792, 0.4054, 0.2153]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

# Train

In [27]:
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [28]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples
):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

In [29]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)
      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

In [30]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')
  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )
  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/10
----------
Train loss 1.0872409107630037 accuracy 0.4228888888888889
Val   loss 0.9459321014583111 accuracy 0.5640000000000001

Epoch 2/10
----------
Train loss 0.8883333366304689 accuracy 0.6164444444444445
Val   loss 0.7578879818320274 accuracy 0.668

Epoch 3/10
----------
Train loss 0.6615071176265104 accuracy 0.764
Val   loss 0.6377465303521603 accuracy 0.784

Epoch 4/10
----------
Train loss 0.57630342160737 accuracy 0.8277777777777778
Val   loss 0.6165517061017454 accuracy 0.8

Epoch 5/10
----------
Train loss 0.49244255673464515 accuracy 0.8655555555555556
Val   loss 0.5782489221310243 accuracy 0.844

Epoch 6/10
----------
Train loss 0.4275978281878954 accuracy 0.8953333333333334
Val   loss 0.5986488443741109 accuracy 0.852

Epoch 7/10
----------
Train loss 0.37297214277842966 accuracy 0.9131111111111112
Val   loss 0.665023560548434 accuracy 0.856

Epoch 8/10
----------
Train loss 0.31712085088767894 accuracy 0.9295555555555556
Val   loss 0.6473916682589334 accuracy 0

# Test Raw Text

In [50]:
example = first_vaccine_data["text"][0:10]
for text in example:
  print("text: " +text)

text: Australia to Manufacture Covid 19 Vaccine and give it to the Citizens for free of cost AFP quotes Prime Minister
text: Australia is doing very good
text: Deaths due to COVID 19 in Affected Countries Read More piryani
text: Subhashree Stay safe di amp da
text: This is what passes for leadership in our country a voucher for something that w
text: The Multi system Inflammatory Syndrome Children MIS C w atypical Kawasaki disease The
text: 88
text: Well let s qualify that would anyone of any party get a vaccine rushed out and minimally tested coming from Russia
text: Most countries without the ability to make locally will be forced to rely on others like the US China
text: zooms up charts in 1st week hear episode


In [51]:
encoded_example = []

for data in example:
  encoded_tweet = tokenizer.encode_plus(
  data,
  max_length=MAX_LEN,
  add_special_tokens=True,
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
)
  encoded_example.append(encoded_tweet)

In [52]:
for encoded_data in encoded_example:
  input_ids = encoded_data['input_ids'].to(device)
  attention_mask = encoded_data['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)

  print(prediction)


tensor([2], device='cuda:0')
tensor([2], device='cuda:0')
tensor([1], device='cuda:0')
tensor([2], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([0], device='cuda:0')
tensor([1], device='cuda:0')


Analsys Data

In [53]:
analysis_data = pd.concat([first_vaccine_data,second_vaccine_data], ignore_index=True)
analysis_data['text'].dropna()
analysis_data.drop_duplicates()
analysis_data.head()

,user_name,user_description,user_location,user_followers,text
0,MyNewsNE,MyNewsNE a dedicated multi-lingual media house...,Assam,64.0,Australia to Manufacture Covid 19 Vaccine and ...
1,Shubham Gupta,I will tell about all experiences of my life f...,NaN,1.0,Australia is doing very good
2,Journal of Infectiology,Journal of Infectiology (ISSN 2689-9981) is ac...,NaN,143.0,Deaths due to COVID 19 in Affected Countries R...
3,Zane,Fresher than you.,NaN,29.0,Subhashree Stay safe di amp da
4,Ann-Maree O’Connor,Retired university administrator. Melburnian b...,"Adelaide, South Australia",83.0,This is what passes for leadership in our coun...


In [54]:
print(analysis_data.shape)

(100000, 5)


In [55]:
sentiment_label = []
encoded_text = []
for text in analysis_data['text']:
  encoded_tweet = tokenizer.encode_plus(
  text,
  max_length=MAX_LEN,
  add_special_tokens=True,
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
)
  encoded_text.append(encoded_tweet)

In [56]:
print(encoded_text[0])

{'input_ids': tensor([[  101,  1754,  1106,  2268, 16205, 11179,  3313,  3291, 18312,  1627,
           159,  7409, 15459,  1105,  1660,  1122,  1106,  1103, 14649,  1111,
          1714,  1104,  2616,   138, 20912, 18328,  3460,  2110,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,

In [57]:
for encoded_data in encoded_text:
  input_ids = encoded_data['input_ids'].to(device)
  attention_mask = encoded_data['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)
  sentiment_label.append(prediction)

In [58]:
print(len(sentiment_label))
print(sentiment_label[0].item())
print(len(encoded_text))

100000
2
100000


In [59]:
sentiment_df = pd.DataFrame(sentiment_label)

# Save to csv

In [60]:
print(sentiment_df)
sentiment_df.to_csv("/content/sentiment.csv",header=False)

                                0
0      tensor(2, device='cuda:0')
1      tensor(2, device='cuda:0')
2      tensor(1, device='cuda:0')
3      tensor(2, device='cuda:0')
4      tensor(1, device='cuda:0')
...                           ...
99995  tensor(2, device='cuda:0')
99996  tensor(2, device='cuda:0')
99997  tensor(1, device='cuda:0')
99998  tensor(2, device='cuda:0')
99999  tensor(2, device='cuda:0')

[100000 rows x 1 columns]


In [61]:
analysis_data.to_csv("/content/anlysis_data.csv")